In [2]:
!pip install --upgrade pip
!pip install salesforce-cdp-connector

  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
salesforce-cdp-connector 1.0.9 requires pip==22.0.4, but you have pip 23.0.1 which is incompatible.
  Using cached pip-22.0.4-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [3]:
# Get sklearn version and make sure that image version supports it
import sklearn
print(sklearn.__version__)
!pip install sagemaker --upgrade
# https://docs.aws.amazon.com/sagemaker/latest/dg/sklearn.html

1.0.2
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from salesforcecdpconnector.connection import SalesforceCDPConnection

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import sagemaker
from sagemaker import get_execution_role

import joblib

import boto3
from botocore.exceptions import ClientError

import json

import os
import time
from time import gmtime, strftime
import subprocess

role = get_execution_role()

# Define input schema for api gateway call
sc = [('engagement_count__c','string')]

sagemaker_session = sagemaker.Session()
#Bucket for model artifacts 
bucket = sagemaker_session.default_bucket()
print(bucket)

project_name = "genie_email_rf_demo"
end_point_prefix = "genie-email-rf-demo"

secret_name = "genie/jdbc"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
region_name = session.region_name

s3 = session.resource('s3')

sagemaker_client = session.client(
    service_name='sagemaker',
    region_name=region_name
)

runtime_client = session.client(
    service_name ='sagemaker-runtime',
    region_name=region_name
)

secretsmanager_client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

try:
    get_secret_value_response = secretsmanager_client.get_secret_value(
        SecretId=secret_name
    )
except ClientError as e:
    # For a list of exceptions thrown, see
    # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    raise e

# Decrypts secret using the associated KMS key.
cred = json.loads(get_secret_value_response['SecretString'])


conn = SalesforceCDPConnection(
        cred['AppUrl'], 
        cred['user'], 
        cred['password'], 
        cred['clientId'], 
        cred['clientSecret']
    )

sagemaker-us-east-1-768394235073


In [5]:
curs = conn.cursor()
query = "SELECT engagement_type__c, engagement_count__c FROM email_engagement_insight__cio limit 900000"
curs.execute(query)
df = pd.DataFrame(curs.fetchall())
df.columns = ['engagement_type__c','engagement_count__c']
df.head(3)

engagement_type__c    engagement_count__c
0               Send  12.000000000000000000
1               Send   2.000000000000000000
2               Open   2.000000000000000000

In [6]:
# Save schema as json
def save_schema(schema):
    res = {}
    sc = []
    for col in schema:
        c = {}
        c['name'] = col[0]
        c['type'] = col[1]
        sc.append(c)
    res['schema']=sc
    json_object = json.dumps(res, indent = 4) 
    with open("schema.json", "w") as outfile:
        outfile.write(json_object)
    
save_schema(sc)

In [7]:
X = df.drop(["engagement_type__c"], axis=1)
y = df["engagement_type__c"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.6813944444444444


In [9]:
# Save model artifacts to S3

with open('model.joblib', 'wb') as f:
    joblib.dump(model,f)

#Build tar file with model data + inference code 
bashCommand = "tar -cvpzf model.tar.gz model.joblib inference.py schema.json"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#Upload tar.gz to bucket
model_artifacts = f"s3://{bucket}/{project_name}/model.tar.gz"
upload_file_response = s3.meta.client.upload_file('model.tar.gz', bucket, project_name+'/model.tar.gz')
print(model_artifacts)

s3://sagemaker-us-east-1-768394235073/genie_email_rf_demo/model.tar.gz


In [10]:
# retrieve sklearn image 
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region_name,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)

In [11]:
#Step 1: Model Creation 
model_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])


Model name: genie-email-rf-demo2023-03-20-00-39-38
Model Arn: arn:aws:sagemaker:us-east-1:768394235073:model/genie-email-rf-demo2023-03-20-00-39-38


In [12]:
#Step 2: EPC Creation 
sklearn_epc_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:768394235073:endpoint-config/genie-email-rf-demo2023-03-20-00-39-39


In [13]:
#Step 3: EP Creation 
endpoint_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

#Monitor creation
describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Endpoint Arn: arn:aws:sagemaker:us-east-1:768394235073:endpoint/genie-email-rf-demo2023-03-20-00-39-39
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'genie-email-rf-demo2023-03-20-00-39-39', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:768394235073:endpoint/genie-email-rf-demo2023-03-20-00-39-39', 'EndpointConfigName': 'genie-email-rf-demo2023-03-20-00-39-39', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:5867d612f7c449d136e6cd7b3a88958bc7754e07290fbd34947a6375c15febb9', 'ResolutionTime': datetime.datetime(2023, 3, 20, 0, 39, 40, 421000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': 

In [14]:
#runtime_client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {"Input": [['2.000000000000000000'],['100.000000000000000000']]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
endpoint_name = describe_endpoint_response['EndpointName']

invoke_endpoint_response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)
result = json.loads(invoke_endpoint_response['Body'].read().decode())['Output']
print(result)

['Send' 'Open']


In [15]:
resource_deletion = True
if resource_deletion:
    sagemaker_client.delete_endpoint(
        EndpointName=endpoint_name
    )
    sagemaker_client.delete_endpoint_config(
        EndpointConfigName=sklearn_epc_name
    )
    sagemaker_client.delete_model(
        ModelName=model_name
    )

In [16]:
Model_Without_Preprocessing_step = True
# Change model group name to any desiered name that starts with key word salesforce
# Change instance type to appropriate instance type
# Define query and schema

model_group_name = 'salesforce-no-preprocess-email-engarement'
instance_type = "ml.m5.xlarge"
query = '''SELECT engagement_type__c, engagement_count__c FROM email_engagement_insight__cio limit 900000'''
schema = {
    'engagement_count__c':'string'
}

def save_metaata():
    dummy_flow = {}
    dummy_flow["nodes"]=[]
    node0={"parameters": {"dataset_definition": {"queryString": query}}}
    node1={"trained_parameters":{"schema":schema}}
    dummy_flow["nodes"].append(node0)
    dummy_flow["nodes"].append(node1)
    return dummy_flow

if Model_Without_Preprocessing_step:
    import os
    import time
    import uuid
    import boto3
    import sagemaker
    from datetime import datetime
    from sagemaker import image_uris
    from sagemaker.model import Model

    # Sagemaker session
    sess = sagemaker.Session()
    region = boto3.Session().region_name
    session = boto3.session.Session(region_name=region)
    sess = sagemaker.Session(boto_session=session)
    iam_role = sagemaker.get_execution_role()
    bucket = sagemaker_session.default_bucket()
    key = f'''custom_metadata/{model_group_name}/dummy.flow'''
    dummy_flow = save_metaata()
    meta_data_json = json.loads(json.dumps(dummy_flow, indent = 4))
    ###### Save Metadata ######
    s3object = s3.Object(bucket, key)
    s3object.put(
        Body=(bytes(json.dumps(meta_data_json).encode('UTF-8')))
    )
    meta_location = f'''s3://{bucket}/{key}'''
    MetadataS3Uri = meta_location
    Model_URI = model_artifacts
    S3MetadataProperties={'flow_file_location': MetadataS3Uri}
    model_name = model_name
    
    model = Model(
        image_uri=image_uri,
        model_data=Model_URI,
        role=iam_role,
        name=model_name,
        sagemaker_session=sess,
        env= {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
    )
    model.register( model_package_group_name=model_group_name,content_types=["application/json"],response_types=["application/json"],
                    inference_instances=[instance_type], approval_status="PendingManualApproval",customer_metadata_properties=S3MetadataProperties)
    


In [17]:
runtime_client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {"Input": [['2.000000000000000000'],['100.000000000000000000']]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
endpoint_name = 'Salesforce-salesforce-no-preprocess-email-engarement-prod'

invoke_endpoint_response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)
result = json.loads(invoke_endpoint_response['Body'].read().decode())['Output']
print(result)

['Send' 'Open']
